In [110]:
# Importing Neccesary Libaries
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC

from xgboost import XGBClassifier
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.neural_network import MLPClassifier

In [111]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

In [112]:
# Generating Correlation Matrix
X_T = np.array(raw_d1)[:,:-1]
corr_mat = pd.DataFrame(np.corrcoef(X_T.T), columns=headers[:-1], index=headers[:-1])
corr_mat.to_csv('Data/Correlation_Matrix.csv')

In [113]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train0, X_cross, Y_train0, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.2, random_state = 42, shuffle = True)
X_train0.shape, Y_train0.shape

((19200, 30), (19200,))

In [114]:
categorical_features = list(range(0,18))

smote = SMOTENC(categorical_features, sampling_strategy='auto', k_neighbors=10, n_jobs=1)
X_train,Y_train =  smote.fit_resample(X_train0, Y_train0)

C:\Users\Janith\.conda\envs\mlbasic\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Janith\.conda\envs\mlbasic\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [107]:
pd.DataFrame(np.concatenate((X_train,np.reshape(Y_train, (Y_train.shape[0], 1))), axis=1)
             ,columns=headers).to_csv('Data/SMOTED.csv', index=None)
X_train.shape, Y_train.shape

((29850, 30), (29850,))

In [97]:
# Standard Normalizing Data
# sc = StandardScaler()
# sc.fit(X_tot)
# X_tot = sc.fit_transform(X_tot)
# X_train = sc.transform(X_train)
# X_cross = sc.transform(X_cross)
# test_data = sc.fit_transform(test_data)

In [108]:
# Applying the Classification Model - Random Forest

# rf = RandomForestClassifier(bootstrap=True, class_weight= None, max_depth= 8, max_features= 10, max_leaf_nodes= None,
#                             min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 100,
#                             n_jobs= 1, oob_score= False, random_state= 85, verbose= 0, warm_start= False)

# rf = XGBClassifier()
rf = GradientBoostingClassifier(n_estimators= 300)

# rf = MLPClassifier(hidden_layer_sizes=(30, 30, 30), activation='tanh', solver='adam', alpha=0.0001, batch_size='auto', 
#                    learning_rate='constant', learning_rate_init=1, power_t=0.5, max_iter=1000, shuffle=True, 
#                    random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, 
#                    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
#                    n_iter_no_change=10, max_fun=15000)

rf.fit(X_train, Y_train)

# Predicting Outcomes
predict_train = rf.predict(X_train)
predict_cross = rf.predict(X_cross)
predict_tot = rf.predict(X_tot)
predict_test = rf.predict(test_data)

In [109]:
# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 50.000%       AUC: 0.5000
Cross Validation Accuracy : 78.021%       AUC: 0.5000 
Total Set Accuracy        : 77.792%       AUC: 0.5000
Confusion Matrix for Cross Validation Set :
 [[3745    0]
 [1055    0]]
Number of Ones Predicted for Test Data : 0
              precision    recall  f1-score   support

         0.0       0.78      1.00      0.88      3745
         1.0       0.00      0.00      0.00      1055

    accuracy                           0.78      4800
   macro avg       0.39      0.50      0.44      4800
weighted avg       0.61      0.78      0.68      4800



C:\Users\Janith\.conda\envs\mlbasic\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/Submission.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')